# 라이브러리

In [14]:
import face_recognition
import cv2
import numpy as np
import glob
import dlib
import time
import pandas as pd

# 함수

In [15]:
font = cv2.FONT_HERSHEY_DUPLEX
try:
    cap = cv2.VideoCapture(0) # 0번 웹캠, 1번 USB 카메라
except:
    print("camera loading error")

def ageGenderPredict():
    age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
    
    age_net = cv2.dnn.readNetFromCaffe(
              'D:/python/age_gender_estimation-master/models/deploy_age.prototxt', 
              'D:/python/age_gender_estimation-master/models/age_net.caffemodel')

    
    img1 = face_recognition.load_image_file("D:/python/data//img1.jpg")
    img1_face_encoding = face_recognition.face_encodings(img1)[0]
    img2 = face_recognition.load_image_file("D:/python/data//img2.jpg")
    img2_face_encoding = face_recognition.face_encodings(img2)[0]
    img3 = face_recognition.load_image_file("D:/python/data//img3.jpg")
    img3_face_encoding = face_recognition.face_encodings(img3)[0]
    
    known_face_encodings = [
        img1_face_encoding,
        img2_face_encoding,
        img3_face_encoding,
    ]

    # 미리 학습시킨 이미지의 이름을 추가
    known_face_names = [
        "LeeJuHyuk",
        "ParkByeongHyun",
        "HeeGyeong",
    ]
    
    face_locations = []#위,아래,좌,우
    face_encodings = []# 벡터모음
    face_name = []#csv에 넣을 이름
    face_age = []#csv에 넣을 나이
    post_data = []
    
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)#거울모드
        #빠른 인식을 위해 얼굴부분을 감지하는 사각형을 1/4로 축소(얼굴크기)
        small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)

        # face_recognition의 인수로 사용하기 위해 BGR 이미지를 RGB로 변경
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        

        # 초기의 배열에 location값과 encoding 값 넣기

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        
        for face_encoding in face_encodings:
            #얼굴이 이미 학습된 얼굴이미지와 같은지 확인
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, 0.4)#T/F
            name = "Unknown"
            #distance가 가장 적은 이미지를 찾아서 이름을 출력(0.6정도가 좋다고 함)
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)#argmin 최솟값 인덱스찾는 속성 1,0
            if matches[best_match_index] == True:
                name = known_face_names[best_match_index]
            face_name.append(name)

            # 두개의 배열을 합치는데 인덱스가 같은 것끼리 합치는 것을 zip
        for (top, right, bottom, left) in face_locations:
            #탐지된 크기가 1/4이였으므로 4배를 다시 키워줌
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # 얼굴 감지하는 사각형 그리기
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)

            # 사각형 밑에 이름 쓰기
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        #(top, right, bottom, left)
        #face_locations = face_recognition.face_locations(rgb_small_frame)
        #x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        for (top, right, bottom, left) in face_locations:
            face_img = frame[top:bottom, left:right]
            blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),
                    mean=(78.4263377603, 87.7689143744, 114.895847746),
                    swapRB=False, crop=False)

            # predict age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_list[age_preds[0].argmax()]
            face_age.append(age)
            
            # visualize
            overlay_text = '%s' % (age)
            cv2.putText(frame, overlay_text, org=(left, top),
              fontFace=font, fontScale=1, color=(255,255,255), thickness=2)
        
        now = time.strftime("%Y_%m_%d_%H_%M_%S")

        cv2.imwrite('D:/python/data/'+now+'.jpg',frame)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
    for face_name, age in zip(face_name, face_age):
        post_data.append((face_name, age))
    dataframe = pd.DataFrame(post_data)
    dataframe.to_csv("D:/python/data/result/" + now + "postdata.csv",header=False, index=False)

# 실행

In [16]:
ageGenderPredict()